In [ ]:
import cv2
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display
import threading
import time
import os

cap = cv2.VideoCapture(0)
image_widget = widgets.Image(format='jpeg', width=400, height=300)
captured_image_widget = widgets.Image(format='jpeg', width=400, height=300)
button_capture = widgets.Button(description="采集图像")

display(image_widget, button_capture, captured_image_widget)

captured_image = None
running = True

# 实时更新摄像头图像的回调函数
def update_camera_image():
    while running:
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            buffer = BytesIO()
            pil_img.save(buffer, format="JPEG")
            image_widget.value = buffer.getvalue()
        time.sleep(0.1)

# 按钮点击事件，采集当前图像并保存
def capture_image(b):
    global captured_image
    ret, frame = cap.read()
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        captured_image = Image.fromarray(frame_rgb)
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        save_path = os.path.expanduser(f'/home/sunrise/Pictures/captured_image_{timestamp}.jpg')
        captured_image.save(save_path, format="JPEG")
        buffer = BytesIO()
        captured_image.save(buffer, format="JPEG")
        captured_image_widget.value = buffer.getvalue()

# 设置按钮点击事件
button_capture.on_click(capture_image)

# 开启一个线程用于实时更新摄像头图像
camera_thread = threading.Thread(target=update_camera_image)
camera_thread.start()

# 当结束时释放摄像头资源
def stop_camera():
    global running
    running = False
    camera_thread.join()
    cap.release()

# 确保程序结束时摄像头释放
import atexit
atexit.register(stop_camera)
